In [2]:
# Cell 2: imports & load dataset
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

CSV_PATH = r"C:\Users\vishn\Documents\HEBA\data\fer2013.csv"  # your path

df = pd.read_csv(CSV_PATH)
print(df.head())
print("Total samples:", len(df))
print("Emotion distribution:\n", df['emotion'].value_counts())


   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
Total samples: 35887
Emotion distribution:
 emotion
3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: count, dtype: int64


In [3]:
# Cell 3: Create PyTorch Dataset loader
from torchvision import transforms

class FERDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        pixels = np.array(self.df.iloc[idx]['pixels'].split(), dtype=np.uint8)
        img = pixels.reshape(48,48)
        
        if self.transform:
            img = self.transform(img)
        
        label = int(self.df.iloc[idx]['emotion'])
        return img, label

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
